In [2]:
# MariaDB에 Python으로 접속하는 방법
import pymysql

In [2]:
# MariaDB 서버 주소와 사용자 정보

HOST = 'stg-yswa-kr-practice-db-master.mariadb.database.azure.com'
USER = 'S09P12A104@stg-yswa-kr-practice-db-master.mariadb.database.azure.com'
PASSWORD = 'l7tRlAa6Dh'
DATABASE = 'S09P12A104'

In [3]:
# 함수 구현


# mariaDB 연결
def connect_to_mariadb(host, user, password, database):
    try:
        dbConnection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            charset='utf8mb4',
            cursorclass=pymysql.cursors.DictCursor
        )
        
        print("MariaDB에 성공적으로 연결되었습니다!")
        return dbConnection

    except pymysql.MySQLError as e:
        print("MariaDB 연결 에러:", e)
        return None


# MariaDB Connection 끊기
def close_connection(dbConnection):
    if dbConnection is not None:
        dbConnection.close()
        print("MariaDB 연결을 닫았습니다.")


# MariaDB 내에 있는 전체 테이블 확인
def show_table_list(dbConnection):
    try:
        cursor = dbConnection.cursor()
        cursor.execute("SHOW TABLES;")
        dbTables = cursor.fetchall()

        # 테이블 목록 출력
        print("현재 데이터베이스에 있는 테이블 목록:")
        for dbTable in dbTables:
            print(dbTable)

        cursor.close()

    except pymysql.Error as e:
        print("쿼리 실행 에러:", e)


# INSERT 기능
#     tableName = 'example_table'
#     insertData = {
#         'column1': 'value1',
#         'column2': 'value2',
#         'column3': 'value3'
#     }

def insert_data(dbConnection, tableName, insertData):
    try:
        cursor = dbConnection.cursor()
        placeholders = ', '.join(['%s'] * len(insertData))
        columns = ', '.join(insertData.keys())
        sql = f"INSERT INTO {tableName} ({columns}) VALUES ({placeholders})"
        cursor.execute(sql, list(insertData.values()))
        dbConnection.commit()
        print("데이터 삽입이 성공적으로 완료되었습니다.")

        cursor.close()

    except pymysql.Error as e:
        dbConnection.rollback()
        print("데이터 삽입 에러:", e)


# Raspberry Pi Serial Number
def get_raspberry_pi_serial():
    try:
        with open('/proc/cpuinfo', 'r') as f:
            for line in f:
                if line.startswith('Serial'):
                    serialNumber = line.split(':')[1].strip()
                    return serialNumber
    except FileNotFoundError:
        print("Error: /proc/cpuinfo file not found.")
        return None
    
    return None

In [ ]:
# 연결 후 테이블 확인

dbConnection = connect_to_mariadb(HOST, USER, PASSWORD, DATABASE)

if dbConnection is not None:
    show_table_list(dbConnection)
    close_connection(dbConnection)